In [7]:
# prompt: 구글드라이브 연동해줘

from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Colab Notebooks/data_analysis_2025/03. 데이터 불러오기

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/data_analysis_2025/03. 데이터 불러오기


### 일반 파일 불러오기 (로컬)

In [8]:
import pandas as pd

In [10]:
csData = pd.read_csv("../dataset/customerdata.csv")

### github 공유된 파일 불러오는것도 가능하다

In [11]:
carDataPath = "https://raw.githubusercontent.com/hyokwan/python-lecture/refs/heads/master/dataset/cars.csv"

In [13]:
carsDf = pd.read_csv(carDataPath)

### 공공데이터포털 데이터 불러오기

In [22]:
pirPath = "https://www.data.go.kr/cmm/cmm/fileDownload.do?atchFileId=FILE_000000002999024&fileDetailSn=1&dataNm=%ED%95%9C%EA%B5%AD%EC%A3%BC%ED%83%9D%EA%B8%88%EC%9C%B5%EA%B3%B5%EC%82%AC_%EC%A3%BC%ED%83%9D%EA%B8%88%EC%9C%B5%EA%B4%80%EB%A0%A8%20%EC%A7%80%EC%88%98_20160101"
pirDf = pd.read_csv( pirPath, encoding = "ms949" )
pirDf.tail(3)

,연도,전국소득대비 주택가격 비율,서울소득대비 주택가격 비율,부산소득대비 주택가격 비율,대구소득대비 주택가격 비율,인천소득대비 주택가격 비율,광주소득대비 주택가격 비율,대전소득대비 주택가격 비율,울산소득대비 주택가격 비율,세종소득대비 주택가격 비율,...,세종평균 대출금액 평균 연소득,경기평균 대출금액 평균 연소득,강원평균 대출금액 평균 연소득,충북평균 대출금액 평균 연소득,충남평균 대출금액 평균 연소득,전북평균 대출금액 평균 연소득,전남평균 대출금액 평균 연소득,경북평균 대출금액 평균 연소득,경남평균 대출금액 평균 연소득,제주평균 대출금액 평균 연소득
140,2015,5.61,7.12,5.54,6.48,5.74,5.36,5.12,5.14,5.16,...,2.39,2.53,1.76,1.90,1.92,1.99,1.64,1.86,2.10,2.64
141,2015,5.68,7.18,5.78,6.48,5.76,5.46,5.17,5.11,5.22,...,2.17,2.56,1.95,2.11,2.02,1.88,1.67,1.99,2.14,2.54
142,2016,4.31,7.30,5.73,6.88,5.74,4.92,4.79,5.38,5.76,...,2.39,2.38,1.74,1.83,1.81,1.63,1.75,1.71,2.05,2.29


### 데이터베이스 데이터 불러오기

In [ ]:
## 08:05 스타트

In [36]:
# !pip install pymysql
# !pip show sqlalchemy

In [50]:
encodingMethod = ["utf-8","ms949"]

In [51]:
for i in range(0,len(encodingMethod)):
    eachEncoding = encodingMethod[i]
    try:
        csData = pd.read_csv("../dataset/join_region_master.csv", encoding = eachEncoding )
    except Exception as e:
        print(e)

'utf-8' codec can't decode byte 0xc7 in position 0: invalid continuation byte


In [52]:
csData

,REGIONID,REGIONNAME
0,A01,한국
1,A02,미국
2,A03,프랑스
3,A04,독일


In [59]:
csData = pd.read_csv("../dataset/customer.csv")
csData["new_col"] = 324
from sqlalchemy import create_engine
### 데이터베이스 접속정보
user = "hkcode"
password = "fintech1308!"
dbip = "my8003.gabiadb.com"
dbport = "3306"
database = "hkcodedb"
dbConPath = f"mysql+pymysql://{user}:{password}@{dbip}:{dbport}/{database}"
dbConPath
myengine = create_engine(dbConPath)
# 데이터 저장하기
csData.to_sql(name = "ANAL_PIRDATA_김효관", con=myengine, index=False, if_exists="replace")

20000

In [60]:
# 데이터 불러오기
csData = pd.read_sql_query(sql="SELECT * FROM ANAL_PIRDATA_김효관", con=myengine)
csData

,balance,stock,label,new_col
0,30000000,22500000,normal,324
1,280000000,48000000,diamond,324
2,300000000,40666666,diamond,324
3,54000000,28000000,normal,324
4,768000000,32000000,vip,324
...,...,...,...,...
19995,628000000,44666666,diamond,324
19996,276000000,20000000,normal,324
19997,652000000,41333333,diamond,324
19998,676000000,45333333,diamond,324
